In [2]:
import sys
sys.path.append('../')
from setting import config_read

In [1]:
from owlready2 import get_ontology
from elasticsearch import Elasticsearch

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [3]:
# Read config file
config = config_read('../')

# Load owl file
data_path = config['owl']['path']
onto = get_ontology(data_path).load()

# Elasticsearch
index_name = config['elasticsearch']['name']
server_ip = config['elasticsearch']['ip']
es = Elasticsearch(server_ip)

In [4]:
query_terms = ['용감한','형제','작곡','노래','장르']

query_patitions = [[" ".join(query_terms)]]

for slice_num in range(1,len(query_terms)):
    for slice_list in itertools.combinations(range(len(query_terms)-1), slice_num):
        qp = []
        qp.append(" ".join(query_terms[:slice_list[0]+1]))
        
        for s_i, slice_idx in enumerate(slice_list):
            if s_i == len(slice_list) - 1 : break
            qp.append(" ".join(query_terms[slice_idx+1:slice_list[s_i+1]+1]))

        qp.append(" ".join(query_terms[slice_list[-1]+1:]))
        query_patitions.append(qp)

print("Query Term Set:", query_terms)
print("Query Partitions Set:")
for qp in query_patitions:
    print(qp)

Query Term Set: ['용감한', '형제', '작곡', '노래', '장르']
Query Partitions Set:
['용감한 형제 작곡 노래 장르']
['용감한', '형제 작곡 노래 장르']
['용감한 형제', '작곡 노래 장르']
['용감한 형제 작곡', '노래 장르']
['용감한 형제 작곡 노래', '장르']
['용감한', '형제', '작곡 노래 장르']
['용감한', '형제 작곡', '노래 장르']
['용감한', '형제 작곡 노래', '장르']
['용감한 형제', '작곡', '노래 장르']
['용감한 형제', '작곡 노래', '장르']
['용감한 형제 작곡', '노래', '장르']
['용감한', '형제', '작곡', '노래 장르']
['용감한', '형제', '작곡 노래', '장르']
['용감한', '형제 작곡', '노래', '장르']
['용감한 형제', '작곡', '노래', '장르']
['용감한', '형제', '작곡', '노래', '장르']


In [5]:
score_list = []

for qp in query_patitions:
    score = 0
    for q in qp:
        search_query = {"query":{"term":{"Annotation Values.keyword": q}}}
        data = es.search(index=index_name, body=search_query)
        if data['hits']['max_score'] :
            score += data['hits']['max_score']
    score_list.append(score)

print(score_list)
p_star = query_patitions[np.argmax(score_list)]
p_star

[0, 0, 7.171753, 0, 6.5522356, 0, 0, 6.5522356, 14.343506, 13.7239886, 13.7239886, 7.171753, 6.5522356, 13.7239886, 28.0674946, 20.8957416]


['용감한 형제', '작곡', '노래', '장르']

In [11]:
candidates_list = []
for q in p_star:
    earch_query = {"query":{"term":{"Annotation Values.keyword": q}}}
    candidates = es.search(index=index_name, body=earch_query)['hits']['hits']
    candidates_list.append(candidates)
resource_combinations = list(itertools.product(*candidates_list))

In [8]:
# Resource Mapping and List Resource Combinations
# candidates_list = [onto.search(label=query) for query in p_star]
# resource_combinations = list(itertools.product(*candidates_list))
# resource_combinations